In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from time import sleep, time

mesh = Mesh(unit_square.GenerateMesh(maxh=0.03))

wind = CF( (y-0.5, 0.5-x))

In [ ]:
order = 3
fesT = L2(mesh, order=order)
fesF = FacetFESpace(mesh, order=order)
fes = fesT*fesF
eT,eF = fes.embeddings
u,uhat = fes.TrialFunction()
v = fesT.TestFunction()
gfu = GridFunction(fes)
gfu2 = GridFunction(fes)

In [ ]:
traceop = fesT.TraceOperator(fesF, average=True)
projinner = Projector(fesF.GetDofs(mesh.Boundaries(".*")), False)
makeboth = (eF@projinner@traceop+eT)@eT.T

In [ ]:
n = specialcf.normal(mesh.dim)
dS = dx(element_vb=BND)
wn = wind*n

convequ = -wind*u*grad(v)*dx + IfPos(wn, wn*u, wn*(2*uhat-u))*v*dS
bf = BilinearForm(convequ, nonlinear_matrix_free_bdb=True, nonassemble=False).Assemble()

# bf1 = BilinearForm(-wind*u*grad(v)*dx, nonlinear_matrix_free_bdb=True).Assemble()
# bf2 = BilinearForm(IfPos(wn, wn*u, wn*(2*uhat-u))*v*dS, nonlinear_matrix_free_bdb=True).Assemble()

massinv = eT@fesT.Mass(1).Inverse() 

In [ ]:
gfu.components[0].Set (exp(-100*((x-0.75)**2+(y-0.5)**2)))

In [ ]:
scene = Draw(gfu.components[0], order=5, euler_angles=(-50,0,-10), deformation=True, scale=0.3)

hv = gfu.vec.CreateVector()
tau = 0.002
t = 0
tend = 10
i = 0
ts = time()
with TaskManager(): # pajetrace=10**8):
    while t < tend:
        t += tau
        # gfu.vec.data -= tau * massinv@bf.mat@makeboth * gfu.vec
        hv.data = gfu.vec - tau/2 * massinv@(bf.mat)@makeboth * gfu.vec
        gfu.vec.data -= tau * massinv@(bf.mat)@makeboth * hv
        i = i+1
        if i%20 == 0:
            scene.Redraw()
print ("time =", time()-ts)

In [ ]:
# print (bf1.mat.GetOperatorInfo())